In [4]:
#!pip install muspy
#!pip install hmmlearn
#muspy.download_musescore_soundfont()

import numpy as np
from tqdm.notebook import tqdm as tqdm
import muspy
from hmmlearn.hmm import CategoricalHMM

**MusPy** is an open source Python library for symbolic music generation. It provides essential tools for developing a music generation system, including dataset management, data I/O,  
data preprocessing and model evaluation.  
**Documentation**: https://salu133445.github.io/muspy/

In [5]:
#List of available datasets in muspy (could also add your own datasets)
#Link: https://salu133445.github.io/muspy/datasets/datasets.html
print(muspy.list_datasets())

[<class 'muspy.datasets.emopia.EMOPIADataset'>, <class 'muspy.datasets.essen.EssenFolkSongDatabase'>, <class 'muspy.datasets.haydn.HaydnOp20Dataset'>, <class 'muspy.datasets.hymnal.HymnalDataset'>, <class 'muspy.datasets.hymnal.HymnalTuneDataset'>, <class 'muspy.datasets.jsb.JSBChoralesDataset'>, <class 'muspy.datasets.lmd.LakhMIDIAlignedDataset'>, <class 'muspy.datasets.lmd.LakhMIDIDataset'>, <class 'muspy.datasets.lmd.LakhMIDIMatchedDataset'>, <class 'muspy.datasets.maestro.MAESTRODatasetV1'>, <class 'muspy.datasets.maestro.MAESTRODatasetV2'>, <class 'muspy.datasets.maestro.MAESTRODatasetV3'>, <class 'muspy.datasets.music21.Music21Dataset'>, <class 'muspy.datasets.musicnet.MusicNetDataset'>, <class 'muspy.datasets.nes.NESMusicDatabase'>, <class 'muspy.datasets.nmd.NottinghamDatabase'>, <class 'muspy.datasets.wikifonia.WikifoniaDataset'>]


Since Baum-Welch is known for its slow convergence, we'll take the lightest dataset available from muspy datasets called the __HaydnOp20__ Dataset consisting of 1.26 hours of recordings \
comprising of 24 classical songs

In [8]:
my_dataset = muspy.datasets.HaydnOp20Dataset(root = '/Users/HassanAdnan/Desktop/PA2.2', download_and_extract=True)
my_dataset = my_dataset.convert()

1114112it [00:00, 6227676.25it/s]         


Successfully downloaded source : /Users/HassanAdnan/Desktop/PA2.2/haydnop20v1.3_annotated.zip .
Extracting archive : /Users/HassanAdnan/Desktop/PA2.2/haydnop20v1.3_annotated.zip ...
Successfully extracted archive : /Users/HassanAdnan/Desktop/PA2.2 .
Converting and saving the dataset...


100%|██████████| 24/24 [00:45<00:00,  1.88s/it]

Successfully saved 15 out of 24 files.


## (HMM + Baum Welch From Scratch) 

Muspy offers all datasets in 4 different representations as mentioned below: 

![Alt text](muspy_representations.png)

Initially, we are only interested in modelling through time and to keep it simple, we'll begin with the __Pitch Representation__. More details here:

![text](muspy_to_pitch.png)

In [9]:
music_collection = []
for music in my_dataset:
    music_collection.append(muspy.to_pitch_representation(music, use_hold_state=True))

### Singing HMM Class
The __Singing_HMM__ Class contains the following methods:

1. `__init__(self, corpus)`: initializes the __POS_HMM__ and prepares it for the parameter initialization phase, contains:
    - a corpus, consisting of unlabeled  sequences of musical units (i.e. all the music songs are flattened and concatenated)
    - a hidden_state_size (default to 10), higher values capture more variability but converge slowly.
    - a tuple of all the unique 
    - a dictionary for mapping the pitches to its unique integer identifier.
    - some additional variables to reduce code redundancy in latter parts such as len()
    - Transition, Emission and Initial State Probability Matrices which are initialized to Zeros.

2. `init_mat(self, init_scheme='uniform')`: __(Can Be Modified)__ initializes the transition, emission and probability matrices either with a 'uniform' value or values sampled randomly from a uniform distribution and normalizes the matrice row wise.

3. `forward(self, sequence)`: __(To Be Implemented)__ implements the Forward stage of the Forward-Backward Algorithm. 
- Feel free to modify function signature and return values.
- Do not change the function name.
4. `backward(self, sequence)`: __(To Be Implemented)__ implements the Forward stage of the Forward-Backward Algorithm. 
- Feel free to modify function signature and return values.
- Do not change the function name.
6. `baum_welch(sequence, alpha, beta)`: __(To Be Implemented)__ implements the Baum Welch Training Algorithm. 
- Feel free to modify function signature and return values.
- Do not change the function name.
7. `softmax(self, x, temperature=1.0)`: calculates the softmax of a given input x adjusting the sharpness of the distribution based on a temperature parameter.

8. `temperature_choice(self, probabilities, temperature=1.0)`: applies a temperature scaling to a set of probabilities and selects an index based on the adjusted probabilities.

9. `sample_sequence(self, length, strategy = "temperature", temperature = 1.0)`: __(Can Be Modified)__ generates a sequence of elements based on a given strategy (probabilistic or temperature) and a specified length. Strategies consists of:
* `probabilistic` strategy:
    -  Samples the initial state based on initial state probabilities.
    -  Iterates over the desired sequence length, sampling an observation based on the current state's emission probabilities, appending the observation to the sequence, and then transitioning to the next state based on the current state's transition probabilities.
* `temperature` strategy:
    -  Similar to the probabilistic strategy but applies temperature scaling to the choice of initial state, observation sampling, and state transitions to adjust the randomness of the choices.

In [137]:
import numpy as np
from scipy.special import logsumexp
from tqdm import tqdm

class Singing_HMM:
    def __init__(self, corpus, hidden_state_size=10):
        self.corpus = [seq.flatten().tolist() for seq in corpus]
        self.hidden_state_size = hidden_state_size
        self.music_seq = [note for seq in self.corpus for note in seq]
        self.vocab = tuple(set(self.music_seq))
        self.vocab2index = {note: i for i, note in enumerate(self.vocab)}
        self.vocab_len = len(self.vocab)
        
        self.transition_mat = np.zeros((self.hidden_state_size, self.hidden_state_size))
        self.emission_mat = np.zeros((self.hidden_state_size, self.vocab_len))
        self.initial_state_prob = np.zeros(self.hidden_state_size)

    def init_mat(self, init_scheme='uniform'): # Can be optionally modified for another initialization scheme (not necessary for the assignment)
        if init_scheme == 'uniform':
            self.transition_mat = np.ones((self.hidden_state_size, self.hidden_state_size))
            self.emission_mat = np.ones((self.hidden_state_size, self.vocab_len))
            self.initial_state_prob = np.ones(self.hidden_state_size)
        elif init_scheme == 'random':
            self.transition_mat = np.random.rand(self.hidden_state_size, self.hidden_state_size)
            self.emission_mat = np.random.rand(self.hidden_state_size, self.vocab_len)
            self.initial_state_prob = np.random.rand(self.hidden_state_size)
        
        self.transition_mat /= self.transition_mat.sum(axis=1, keepdims=True)
        self.emission_mat /= self.emission_mat.sum(axis=1, keepdims=True)
        self.initial_state_prob /= self.initial_state_prob.sum()
    
    def forward(self, sequence):
        """
        Forward algorithm for calculating the probabilities of a sequence.
        """
        seq_len = len(sequence)
        alpha = np.random.randn(self.hidden_state_size, seq_len)
        
        # Initialization
        for i in range(self.hidden_state_size):
            alpha[i, 0] = np.log(self.initial_state_prob[i]) + np.log(self.emission_mat[i, self.vocab2index[sequence[0]]])
        
        # Induction
        for t in range(1, seq_len):
            for j in range(self.hidden_state_size):
                log_alpha_sum = logsumexp(alpha[:, t-1] + np.log(self.transition_mat[:, j]))
                alpha[j, t] = log_alpha_sum + np.log(self.emission_mat[j, self.vocab2index[sequence[t]]])
        
        # Debugging print statements
        print("Alpha:")
        print(alpha)
        
        return alpha


    def backward(self, sequence):
        """
        Backward algorithm for calculating the probabilities of a sequence.
        """
        T = len(sequence)
        beta = np.random.randn(self.hidden_state_size, T)

        # Initialization step
        for s in range(self.hidden_state_size):
            beta[s, T-1] = 0

        # Recursion step
        for t in range(T-2, -1, -1):
            for s in range(self.hidden_state_size):
                log_beta_sum = logsumexp(np.log(self.transition_mat[s, :]) + np.log(self.emission_mat[:, self.vocab2index[sequence[t+1]]]) + beta[:, t+1])
                beta[s, t] = log_beta_sum
        
        # Normalize beta values
        beta -= logsumexp(beta)
        beta = np.exp(beta)

        return beta
    
    def baum_welch(self, n_iter=100, tol=1e-4):
        """
        Perform Baum-Welch training to update the model's parameters.
        """

        prev_log_likelihood = float('-inf')  # Initialize with negative infinity (DO NOT CHANGE THIS VARIABLE)

        for iteration in tqdm(range(n_iter), desc="Training Progress", leave=True):
            log_likelihood = 0 # Log likelihood for this iteration (DO NOT CHANGE THIS VARIABLE)
            alpha_list = []
            beta_list = []
            xi_list = []
            gamma_list = []

            for seq in self.corpus:
                seq_len = len(seq)
                alpha = self.forward(seq)
                beta = self.backward(seq)
                alpha_list.append(alpha)
                beta_list.append(beta)
                
                # Compute xi and gamma
                xi = np.zeros((seq_len - 1, self.hidden_state_size, self.hidden_state_size))
                gamma = np.zeros((seq_len, self.hidden_state_size))
                for t in range(seq_len - 1):
                    for i in range(self.hidden_state_size):
                        for j in range(self.hidden_state_size):
                            xi[t, i, j] = alpha[i, t] + np.log(self.transition_mat[i, j]) + np.log(self.emission_mat[j, self.vocab2index[seq[t+1]]]) + beta[j, t+1]
                    xi[t] -= logsumexp(xi[t])
                    gamma[t] = logsumexp(xi[t], axis=1)
                
                for i in range(self.hidden_state_size):
                    gamma[seq_len-1, i] = alpha[i, seq_len-1] + beta[i, seq_len-1]
                gamma[seq_len-1] -= logsumexp(gamma[seq_len-1])

                xi_list.append(xi)
                gamma_list.append(gamma)
                
                log_likelihood += logsumexp(alpha[:, seq_len-1])

                # Print values for debugging
                print("Alpha:")
                print(alpha)
                print("Beta:")
                print(beta)
                print("Xi:")
                print(xi)
                print("Gamma:")
                print(gamma)

            # Update transition matrix, emission matrix, and initial state probability
            for i in range(self.hidden_state_size):
                self.initial_state_prob[i] = np.mean([np.exp(gamma[0, i]) for gamma in gamma_list])
                for j in range(self.hidden_state_size):
                    numerator = np.sum([np.exp(xi[t, i, j]) for xi in xi_list for t in range(len(xi))])
                    denominator = np.sum([np.exp(gamma[t, i]) for gamma in gamma_list for t in range(len(gamma)-1)])
                    self.transition_mat[i, j] = numerator / denominator
                    
            for j in range(self.hidden_state_size):
                for k in range(self.vocab_len):
                    numerator = 0
                    denominator = 0
                    for seq, gamma in zip(self.corpus, gamma_list):
                        for t in range(len(seq)):
                            if seq[t] == self.vocab[k]:
                                numerator += np.exp(gamma[t, j])
                            denominator += np.exp(gamma[t, j])
                    self.emission_mat[j, k] = numerator / denominator
            
            # Print log likelihood for debugging
            print(f"Iteration {iteration + 1}: Log Likelihood: {log_likelihood}")


            if iteration == 0:
                convergence_rate = convergence_diff = np.nan  # Print nan for the first iteration
            else:
                convergence_diff = np.abs(log_likelihood - prev_log_likelihood)
                convergence_rate = convergence_diff / np.abs(prev_log_likelihood)
            
            #Note that Log Likelihoods would be negative and would increase (i.e. go in the direction of 0) as the model converges.
            # Log Likelihoods may be far from 0, but the increasing trend should remain present.
            tqdm.write(f"Iteration {iteration + 1}: Log Likelihood: {log_likelihood}, Convergence Difference: {convergence_diff} , Convergence Rate: {convergence_rate}")
            
            if iteration > 0 and convergence_rate < tol:
                tqdm.write("Convergence achieved.")
                break
            
            prev_log_likelihood = log_likelihood



    def softmax(self, x, temperature=1.0):
        '''Compute softmax values for each set of scores in x.'''
        e_x = np.exp((x - np.max(x)) / temperature)
        return e_x / e_x.sum()

    def temperature_choice(self, probabilities, temperature=1.0):
        '''Apply temperature to probabilities and make a choice.'''
        adjusted_probs = self.softmax(np.log(probabilities + 1e-9), temperature)  # Adding epsilon to avoid log(0)
        return np.random.choice(len(probabilities), p=adjusted_probs)
                        
    def sample_sequence(self, length, strategy = "temperature", temperature = 1.0):
        sequence = []
        if strategy == 'probabilistic':
            # Sample the initial state
            state = np.random.choice(self.hidden_state_size, p=self.initial_state_prob)
            for _ in range(length):
                # Sample an observation (note) based on the current state
                note = np.random.choice(self.vocab, p=self.emission_mat[state])
                sequence.append(note)
                # Transition to the next state
                state = np.random.choice(self.hidden_state_size, p=self.transition_mat[state])
        elif strategy == 'temperature':
            # Sample the initial state with temperature
            state = self.temperature_choice(self.initial_state_prob, temperature)
            for _ in range(length):
                # Apply temperature to emission probabilities and sample a note
                note = self.temperature_choice(self.emission_mat[state], temperature)
                sequence.append(self.vocab[note])
                # Transition to the next state with temperature
                state = self.temperature_choice(self.transition_mat[state], temperature)
        return sequence

In [138]:
#Specify values and run the code to test your implementation
pos_hmm = Singing_HMM(corpus=music_collection, hidden_state_size=10)
pos_hmm.init_mat(init_scheme='uniform')  # Choose the initialization scheme
pos_hmm.baum_welch(tol=1e-4, n_iter=100)  # Set the tolerance and number of iterations


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Alpha:
[[-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 ...
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]]
Alpha:
[[-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 [-6.39692966e+00 -1.04912742e+01 -1.45856188e+01 ... -4.27390711e+04
  -4.27431655e+04 -4.27472598e+04]
 ...
 [-6.39692966e+00 -1.04912742e

Training Progress:   1%|          | 1/100 [01:21<2:14:15, 81.37s/it]

Iteration 1: Log Likelihood: -474616.42165279615
Iteration 1: Log Likelihood: -474616.42165279615, Convergence Difference: nan , Convergence Rate: nan
Alpha:
[[-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 ...
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]]
Alpha:
[[-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.260

Training Progress:   2%|▏         | 2/100 [02:44<2:14:08, 82.12s/it]

Iteration 2: Log Likelihood: -98570.16747590058
Iteration 2: Log Likelihood: -98570.16747590058, Convergence Difference: 376046.2541768956 , Convergence Rate: 0.7923161463047539
Alpha:
[[-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 ...
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]]
Alpha:
[[-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+03
  -9.26040518e+03 -9.26055876e+03]
 [-7.80626168e+00 -7.95983886e+00 -8.11341603e+00 ... -9.26025160e+

Training Progress:   2%|▏         | 2/100 [04:05<3:20:11, 122.56s/it]

Iteration 3: Log Likelihood: -98570.16747590699
Iteration 3: Log Likelihood: -98570.16747590699, Convergence Difference: 6.402842700481415e-09 , Convergence Rate: 6.495720626676266e-14
Convergence achieved.


In [139]:
notes_seq = pos_hmm.sample_sequence(1024, strategy= "probabilistic") #Feel free to experiment with the sampling strategy
synthetic_music = muspy.from_pitch_representation(np.array(notes_seq), resolution=24, program=0, is_drum=False, use_hold_state=True, default_velocity=64)
muspy.write_midi('/Users/HassanAdnan/Desktop/PA2.2/pitch_based.mid', synthetic_music) #Specify the path to save the MIDI file, name it "pitch_based.mid"

## Synthetic Music Generation via __HMMLearn__ 

#### __Note:__ For any model that you train/fit, remember to set __verbose = True__

In [97]:
hidden_states = 32 #Number of hidden states in the HMM model (Feel free to change or experiment with this value)
sythetic_music_sequence_length = 128 #Length of the synthetic music sequence to be generated (could be either a time step, an event or a note)

In [160]:
import muspy
import numpy as np
from hmmlearn import hmm



# Define the model parameters
num_components = 5  # Number of hidden states
num_iterations = 500  # Number of iterations for training
tolerance = 0.01  # Tolerance for convergence
resolution = 24  # MIDI resolution
program = 0  # MIDI program number (instrument)
is_drum = False  # Whether it's a drum track
use_start_end = False  # Whether to use start and end tokens in the note representation
encode_velocity = True  # Whether to encode note velocity
default_velocity = 64  # Default velocity if velocity is not encoded in the note representation

# Flatten the music collection to fit the model
flattened_music = np.concatenate([muspy.to_note_representation(music, use_start_end=use_start_end, encode_velocity=encode_velocity, dtype=int) for music in music_collection])

# Initialize and fit a Gaussian HMM model
model = hmm.GaussianHMM(n_components=num_components, covariance_type="diag", n_iter=num_iterations, tol=tolerance, verbose=True)
model.fit(flattened_music)




         1 -344571.62781699             +nan
         2 -257243.56521957  +87328.06259743
         3 -251714.57273106   +5528.99248851
         4 -248156.84404304   +3557.72868802
         5 -244345.89726404   +3810.94677900
         6 -241938.56739974   +2407.32986430
         7 -241018.74484683    +919.82255290
         8 -240582.43638630    +436.30846053
         9 -240402.85819967    +179.57818663
        10 -240223.07448173    +179.78371794
        11 -240037.60222387    +185.47225786
        12 -239874.49929025    +163.10293362
        13 -239702.53631109    +171.96297916
        14 -239596.12201118    +106.41429991
        15 -239499.30232619     +96.81968499
        16 -239440.89044995     +58.41187623
        17 -239418.31670686     +22.57374310
        18 -239406.72274796     +11.59395890
        19 -239398.94233689      +7.78041106
        20 -239392.43343055      +6.50890634
        21 -239385.78302650      +6.65040406
        22 -239373.34115697     +12.44186953
        23

GaussianHMM(n_components=5, n_iter=500, verbose=True)

In [161]:
#Sampling a sequence of music from the model and save as a MIDI file, name it "event_based.mid"
# Generate a sequence of notes
generated_sequence = model.sample(500)[0]  # Generate 500 notes

# Round to nearest integers and ensure non-negative times
generated_sequence = np.rint(generated_sequence).astype(int)
generated_sequence[:, 0] = np.abs(generated_sequence[:, 0])  # Ensure time is non-negative

# Sort the notes by time
generated_sequence = generated_sequence[generated_sequence[:, 0].argsort()]

# Convert the generated sequence back to a muspy Music object
generated_music = muspy.from_note_representation(generated_sequence, resolution=resolution, program=program, is_drum=is_drum, use_start_end=use_start_end, encode_velocity=encode_velocity, default_velocity=default_velocity)

# Save the generated music as a MIDI file
muspy.write_midi("event_based.mid", generated_music)

In [162]:
#Write your code here and save the sampled sequence as MIDI file, name it "note_based.mid"z
 

import muspy
import numpy as np
from hmmlearn import hmm

music_collection = []
for music in my_dataset:  
    note_representation = muspy.to_note_representation(music, use_start_end=False, encode_velocity=True, dtype=int)
    music_collection.append(note_representation)
    
model = hmm.GaussianHMM(n_components=5, covariance_type="diag", n_iter=500, tol=0.01, verbose=True)
# Flatten your music collection to fit the model
flattened_music = np.concatenate(music_collection)
model.fit(flattened_music)


generated_sequence = model.sample(500)[0]  

# Round to nearest integers and ensure non-negative times
generated_sequence = np.rint(generated_sequence).astype(int)
generated_sequence[:, 0] = np.abs(generated_sequence[:, 0])  

# Sort the notes by time
generated_sequence = generated_sequence[generated_sequence[:, 0].argsort()]

generated_music = muspy.from_note_representation(generated_sequence, resolution=24, program=0, is_drum=False, use_start_end=False, encode_velocity=True, default_velocity=64)

muspy.write_midi("note_based.mid", generated_music)

         1 -348828.55771630             +nan
         2 -257948.01570948  +90880.54200682
         3 -249533.74681378   +8414.26889570
         4 -245888.62038725   +3645.12642653
         5 -243899.25739471   +1989.36299254
         6 -241033.60020256   +2865.65719215
         7 -238484.58692622   +2549.01327634
         8 -237991.77246590    +492.81446032
         9 -237786.60991051    +205.16255539
        10 -237684.25909794    +102.35081257
        11 -237643.59626301     +40.66283494
        12 -237631.02591439     +12.57034862
        13 -237626.74109241      +4.28482198
        14 -237624.04785829      +2.69323411
        15 -237621.56442010      +2.48343819
        16 -237617.57825636      +3.98616375
        17 -237610.74874255      +6.82951381
        18 -237599.13154229     +11.61720026
        19 -237574.37387504     +24.75766725
        20 -237548.88255273     +25.49132231
        21 -237527.40979944     +21.47275329
        22 -237503.66099365     +23.74880579
        23